## Upload Data

In [16]:
# run this cell
import syft as sy
from utils import *
from syft.core.adp.data_subject_list import DataSubjectArray
from tqdm import tqdm

In [6]:
# Get dataset splits

dataset_names = []
path = "https://raw.githubusercontent.com/OpenMined/datasets/main/BreastCancerDataset/subsets/"
with open("dataset_names.txt", "r") as f:
    for line in f:
        dataset_names.append(path + line.strip())

print(len(dataset_names))

100


In [17]:
# Get IP addresses for domains
domain_ips = []
pwds = []
with open("IPLogin.csv", "r") as f:
    for line in f:
        struct = line.split(",")
        domain_ips.append(struct[0])
        pwds.append(struct[-1])
        

domain_ips = domain_ips[:100]
pwds = pwds[:100]
print(len(domain_ips), len(domain_ips))

100 100


In [ ]:
for i in tqdm(range(100)):
    
    # Need to run update_syft.sh on every domain
    sshpass -p MflpTeamOne@31052022 ssh azureuser@20.253.155.189 update_syft.sh
    
    # Log into domain
    domain = sy.login(email="info@openmined.org", password="changethis",url=domain_ips[i])
    
    # Preprocess dataset split
    dataset = download_dataset(dataset_names[i])
    train, val, test = split_and_preprocess_dataset(data=dataset)
    
    data_subjects_image = np.ones(train["images"].shape).astype(object)
    for i,patient in enumerate(train["patient_ids"]):
        data_subjects_image[i] = DataSubjectArray([str(patient)])

    data_subjects_labels = np.ones(train["labels"].shape).astype(object)
    for i,patient in enumerate(train["patient_ids"]):
        data_subjects_labels[i] = DataSubjectArray([str(patient)])


    train_image_data = sy.Tensor(train["images"]).annotated_with_dp_metadata(
        min_val=0, max_val=255, data_subjects=data_subjects_image
    )
    train_label_data = sy.Tensor(train["labels"]).annotated_with_dp_metadata(
        min_val=0, max_val=1, data_subjects=data_subjects_labels
    )
    
    # Load dataset
    domain.load_dataset(
        name="BreastCancerDataset",
        assets={
            "train_images": train_image_data[:2],
            "train_labels": train_label_data[:2],
        },
        description="Invasive Ductal Carcinoma (IDC) is the most common subtype of all breast cancers. \
        The modified dataset consisted of 162 whole mount slide images of Breast Cancer (BCa) specimens scanned at 40x. \
        Patches of size 50 x 50 were extracted from the original image. The labels 0 is non-IDC and 1 is IDC."
    )
    
    domain.create_user(
        name="Sam Carter",
        email="sam@stargate.net",
        password="changethis",
        budget=9999
    )
